In [2]:
import shapefile

In [3]:
sf = shapefile.Reader("world/data/VGD-Oesterreich_gen_250/BEV_VGD_250_LAM_mitAttributen_shp_2016_10_02.shp")

In [4]:
sf.fields

[('DeletionFlag', 'C', 1, 0),
 ['KG_NR', 'C', 6, 0],
 ['KG', 'C', 50, 0],
 ['MERIDIAN', 'C', 3, 0],
 ['GKZ', 'C', 6, 0],
 ['PG', 'C', 50, 0],
 ['BKZ', 'C', 4, 0],
 ['PB', 'C', 50, 0],
 ['FA_NR', 'C', 3, 0],
 ['FA', 'C', 50, 0],
 ['GB_KZ', 'C', 4, 0],
 ['GB', 'C', 50, 0],
 ['VA_NR', 'C', 3, 0],
 ['VA', 'C', 50, 0],
 ['BL_KZ', 'C', 2, 0],
 ['BL', 'C', 50, 0],
 ['ST_KZ', 'N', 2, 0],
 ['ST', 'C', 50, 0],
 ['FL', 'N', 12, 0]]

In [5]:
shaperecs = sf.shapeRecords()

In [14]:
from django.contrib.gis.geos import Polygon, MultiPolygon

In [41]:
shaperecs[0].shape.points

[(413199.7849999964, 512448.1479999982),
 (413255.9849999994, 512448.52600000054),
 (413254.49099999666, 512609.4059999995),
 (413110.93299999833, 513015.9439999983),
 (413423.5319999978, 512970.9210000001),
 (413642.3189999983, 513176.21400000155),
 (413929.11100000143, 513061.07499999925),
 (413896.186999999, 513326.1160000004),
 (413792.58200000226, 513469.3740000017),
 (414211.16099999845, 513914.4439999983),
 (414306.7950000018, 514206.60599999875),
 (414746.18599999696, 514256.3189999983),
 (414904.5329999998, 514163.7800000012),
 (415278.0719999969, 514129.01000000164),
 (415654.66099999845, 513888.9780000001),
 (415811.1730000004, 513913.3370000012),
 (415576.01099999994, 513544.73699999973),
 (415727.41600000113, 513388.48900000006),
 (415567.97299999744, 513093.6009999998),
 (415592.58900000155, 512639.60599999875),
 (415398.09799999744, 512468.06700000167),
 (415477.6960000023, 512311.36400000006),
 (415777.50599999726, 512240.53400000185),
 (415837.54100000113, 512147.26000

In [62]:
from datetime import date

In [63]:
src = Source(name='BEV Österreich, VGD', original_url="http://www.bev.gv.at/pls/portal/docs/PAGE/BEV_PORTAL_CONTENT_ALLGEMEIN/0200_PRODUKTE/UNENTGELTLICHE_PRODUKTE_DES_BEV/VGD-Oesterreich_gen_250.zip", downloaded=date.today())

In [64]:
src.save()

/home/vagrant/.venvs/geodjango/lib/python3.5/site-packages/django/db/models/fields/__init__.py:1393: RuntimeWarning: DateTimeField Source.downloaded received a naive datetime (2017-01-13 00:00:00) while time zone support is active.
  RuntimeWarning)



In [66]:
sf_fields = sf.fields

In [69]:
sf_fields.pop(0)

('DeletionFlag', 'C', 1, 0)

In [70]:
sf_fields

[['KG_NR', 'C', 6, 0],
 ['KG', 'C', 50, 0],
 ['MERIDIAN', 'C', 3, 0],
 ['GKZ', 'C', 6, 0],
 ['PG', 'C', 50, 0],
 ['BKZ', 'C', 4, 0],
 ['PB', 'C', 50, 0],
 ['FA_NR', 'C', 3, 0],
 ['FA', 'C', 50, 0],
 ['GB_KZ', 'C', 4, 0],
 ['GB', 'C', 50, 0],
 ['VA_NR', 'C', 3, 0],
 ['VA', 'C', 50, 0],
 ['BL_KZ', 'C', 2, 0],
 ['BL', 'C', 50, 0],
 ['ST_KZ', 'N', 2, 0],
 ['ST', 'C', 50, 0],
 ['FL', 'N', 12, 0]]

In [109]:
errors_shapes = []

In [110]:
for x in shaperecs:
    try:
        p = Polygon(x.shape.points, srid=31287)
    except:
        errors_shapes.append(x)
        continue
    p.transform(4326)
    p2 = MultiPolygon(p)
    lst_legacy = dict()
    for indx, f in enumerate(sf_fields):
        if type(x.record[indx]) == bytes:
            entry = x.record[indx].decode('latin1')
        else:
            entry = str(x.record[indx])
        lst_legacy[f[0]] = entry
    if type(x.record[1]) == bytes:
        name = x.record[1].decode('latin1')
    else:
        name = x.record[1]
    a = Area(geom=p2, name=name, legacy_properties=lst_legacy, source=src)
    a.save()

GEOS_ERROR: IllegalArgumentException: Points of LinearRing do not form a closed linestring



In [113]:
g = Area.objects.get(pk=4965)

In [114]:
g.legacy_properties['BKZ']

'414'

In [112]:
errors_shapes[0].record

['85103',
 'Matrei in Osttirol Land',
 'M31',
 '70717',
 'Matrei in Osttirol',
 '707',
 'Lienz',
 '082',
 b'Kitzb\xfchel Lienz',
 '850',
 'Lienz',
 '85',
 'Lienz',
 '7',
 'Tirol',
 1,
 b'\xd6sterreich',
 275388233]